#### 네이버 뉴스 sub페이지 크롤링
   - 한개의 subpage 크롤링 한 후 함수로 재구성
   - 02_1번에서 수집된 데이터를 이용해서 여러 subpage 크롤링

In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [2]:
# 뉴스 subpage 문서 로드
url = 'https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=101&oid=008&aid=0004626352'
headers = {"user-Agent":
           "Mozilla/5.0(Window NT 10.0;Win64;x64) AppleWebKit/537.36 (KHTML,likeGecko) Chrome/87.0.4280.88 Safari/537.36"
}
result = requests.get(url,headers=headers)

In [3]:
result

<Response [200]>

In [4]:
# result.content # 전송된 소스코드를 반환 raw text 형식으로 반환 앞에 b코드가 붙어있음
# result.text # 전송된 소스코드를 분자열로 반환

In [5]:
bs_obj = BeautifulSoup(result.content,"html.parser")
# 기사 제목 추가
title = bs_obj.find(id='articleTitle')
title.text
# 기사 입력 시간
time = bs_obj.find("span",{"class":"t11"})
time.text


'2021.08.04. 오전 11:31'

#### 개발자 도구의 copy selector 사용 방법

- 크롤링하려는 컨텐츠에서 검사 - 해당 소스에서 단축메뉴 - copy - copy selector
    - 붙여넣기 하면 선택자를 반환해주는데 선택된 내용이 없는 경우가 있음
        - 잘못된 선택자를 생성했을 경우 - 선택자 재구성
        - 동적으로 생성되는 태그인 경우에는 선택자가 만들어지지만 정적 크롤링 불가능
        - 네이버 뉴스 서브페이지의 좋아요 수와 기사 댓글 수는 동적 생성되어서 정적 크롤링 불가능
        - 셀레니움이라는 패키지를 사용해야 함

- bs_obj.select("#main_content > div.article_header > div.article_info > div > div.article_btns > div.article_btns_left > div > a > span.u_likeit_text._count.num")


In [6]:
# sub페이지에서 수집된 데이터 반환하는 함수 작성(서브페이지 URL을 파라미터로 전달받음)

def get_news_sub_info(url):
    
    # 1. 전달된 url에 접속 후 응답객체 반환
    headers = {"user-Agent":
           "Mozilla/5.0(Window NT 10.0;Win64;x64) AppleWebKit/537.36 (KHTML,likeGecko) Chrome/87.0.4280.88 Safari/537.36"}
    result = requests.get(url,headers=headers)
    # 2. bs4 객체 생성
    bs_obj = BeautifulSoup(result.content,"html.parser")
    
    # 3. 필요 데이터 추출
    # 기사 제목 추출(태그 객체)
    title = bs_obj.find(id='articleTitle')
    # 기사 입력 시간 추출(태그 객체)
    time = bs_obj.find("span",{"class":"t11"})
    # 기사 내용 추출
    content = bs_obj.find(id='articleBodyContents')
    
    # 4. 수집 데이터 dict로 구성 후 반환
    dict_sub = {}
    dict_sub["기사제목"] = title.text
    dict_sub["기사 입력 시간"] = time.text
    dict_sub["기사 내용"] = content.text.strip()
    
    return dict_sub

In [7]:
## main 함수호출 code

headers = {"user-Agent":"Mozilla/5.0(Window NT 10.0;Win64;x64) AppleWebKit/537.36 (KHTML,likeGecko) Chrome/87.0.4280.88 Safari/537.36"
}

url = "https://news.naver.com/"
res = requests.get(url,headers=headers)
html = res.text
bs_obj = BeautifulSoup(html,"html.parser")

# 헤드라인 뉴스의 구성 ul, class : hdline_article_list
news_list = bs_obj.find("ul", {"class":"hdline_article_list"})
# news_list

# 모든 li 추출
lis = news_list.findAll("li")

link_list=[]
for li in lis:
    a_tag = li.find("a")
    link_list.append("https://news.naver.com/"+a_tag["href"])
# link_list
for url in link_list:
    res = get_news_sub_info(url)
    print(res)

{'기사제목': '내년 최저임금 시급 9160원으로 확정…월 191만 4천 원', '기사 입력 시간': '2021.08.05. 오전 6:33', '기사 내용': '동영상 뉴스\n\n\n\n\n\t\n\t[앵커]내년 최저임금이\xa0시급 9,160원으로 확정됐습니다.\xa0경제단체들의 이의 제기는\xa0결국 받아들여지지 않았는데요.\xa0한국경영자총협회 등은\xa0강한 유감을 표시했습니다.\xa0장지현 기자,\xa0내년 최저임금이 확정됐군요?\xa0[기자]네, 내년도 최저임금이 시간당 9,160원으로 올해 최저임금 8,720원보다 5.1%,\xa0그러니까 440원 올랐습니다.\xa0이에 따라 내년도 최저임금은 내년 1월 1일부터 적용됩니다.\xa0고용노동부 확정 고시에는\xa0월 노동시간 209시간을 적용한 내년 최저임금 월 환산액 191만 4,440원도 병기됐고요.\xa0또 업종과 상관없이 모든 사업장에\xa0동일하게 적용한다는 점도 명시됐습니다.\xa0[앵커]경제단체와 소상공인들이\xa0굉장히 반발했던 것으로 아는데,\xa0결국 이의제기가 받아들여지지 않은 거네요?[기자]네, 앞서 최저임금위원회는\xa0지난달 12일 내년도 최저임금을 9,160원으로 의결했는데요.\xa0경총과 중기중앙회, 소상공인연합회 등 사용자단체는 이에 대해 이의를 제기했습니다.\xa0고용노동부가 최저임금안을 고시하기 전에\xa0노사 단체는 이의를 제기할 수 있고, 고용부는 이의가 합당하다고 인정되면\xa0최저임금위원회에 재심의를 요청할 수 있습니다.\xa0하지만 최저임금 제도가 도입된 1988년 이후\xa0노사의 이의 제기는 20여 차례 있었지만, 재심의가 이뤄진 적은 없었고,\xa0이번에도 역시 마찬가지였습니다.\xa0경총은 노동부가 이의 제기를\xa0수용하지 않은 데 대해 "5.1% 인상은\xa0이미 한계에 놓인\xa0중소·영세기업과 소상공인이\xa0도저히 감당하기 어려운 수준"이라며\xa0반발했습니다.SBS Biz 장지현입니다.'}
{'기사제목': '대구 교회·태권도장서 하루 만에

In [ ]:
## 연습문제
## get_news_sub_info 함수의 반환값에 기사 내용까지 포함되게 코드를 수정하고
## 반환되는 결과값을 df로 구성하시오

In [8]:
# sub페이지에서 수집된 데이터 반환하는 함수 작성(서브페이지 URL을 파라미터로 전달받음)

def get_news_sub_info(url):
    
    # 1. 전달된 url에 접속 후 응답객체 반환
    headers = {"user-Agent":
           "Mozilla/5.0(Window NT 10.0;Win64;x64) AppleWebKit/537.36 (KHTML,likeGecko) Chrome/87.0.4280.88 Safari/537.36"}
    result = requests.get(url,headers=headers)
    # 2. bs4 객체 생성
    bs_obj = BeautifulSoup(result.content,"html.parser")
    
    # 3. 필요 데이터 추출
    # 기사 제목 추출(태그 객체)
    title = bs_obj.find(id='articleTitle')
    # 기사 입력 시간 추출(태그 객체)
    time = bs_obj.find("span",{"class":"t11"})
    # 기사 내용 추출
    content = bs_obj.find(id='articleBodyContents')
    
    # 4. 수집 데이터 dict로 구성 후 반환
    dict_sub = {}
    dict_sub["기사제목"] = title.text
    dict_sub["기사 입력 시간"] = time.text
    dict_sub["기사 내용"] = content.text.strip()
    
    return dict_sub

- 함수 끝

#### main 시작

In [11]:
# 빈 데이터 프레임 하나 생성
import pandas as pd
dict_sub = {}
dict_sub["기사제목"] = []
dict_sub["기사 입력 시간"] = []
dict_sub["기사 내용"] = []
news_sub = pd.DataFrame(dict_sub)
news_sub

,기사제목,기사 입력 시간,기사 내용


In [12]:
## main 함수호출 code

headers = {"user-Agent":"Mozilla/5.0(Window NT 10.0;Win64;x64) AppleWebKit/537.36 (KHTML,likeGecko) Chrome/87.0.4280.88 Safari/537.36"
}

url = "https://news.naver.com/"
res = requests.get(url,headers=headers)
html = res.text
bs_obj = BeautifulSoup(html,"html.parser")

# 헤드라인 뉴스의 구성 ul, class : hdline_article_list
news_list = bs_obj.find("ul", {"class":"hdline_article_list"})
# news_list

# 모든 li 추출
lis = news_list.findAll("li")

link_list=[]
dict_news = {}
for li in lis:
    a_tag = li.find("a")
    link_list.append("https://news.naver.com/"+a_tag["href"])
# link_list
for url in link_list:
    res = get_news_sub_info(url)
    # 빈 df에 반환되는 dict 행으로 추가(append 함수 이용)
    news_sub = news_sub.append(res, ignore_index = True)
    
news_sub
    

,기사제목,기사 입력 시간,기사 내용
0,프리미엄 노트북 'LG그램 블랙 라벨' 출시…319만원,2021.08.05. 오전 10:00,LG 그램 한정판 'LG 그램 블랙 라벨' /사진=LG전자LG전자가 초경량 노트북 ...
1,'음주운전·무능' 李·李 또 충돌…선 넘은 비방전도,2021.08.05. 오전 5:37,"동영상 뉴스\n\n\n\n\n[앵커]더불어민주당의 대선 경선 두번째 토론회에서, 이..."
2,[내일날씨] 서울 35도·강릉 36도 '더위 계속'…내륙 5~40㎜ 소나기,2021.08.05. 오전 9:00,"아침 22~27도, 낮 31~36도…체감 35도 웃도는 곳도경기 북부 내륙·강원 내..."
3,분당 김밥집 식중독 134명… 본사 재료 탓?,2021.08.05. 오전 5:07,2개 지점서 각 이틀간 1100여명 구매당시 재료 소진… 칼·도마 등 역학조사김밥 ...
4,"[클릭 e종목]""한국타이어앤테크놀로지, 가격 전가도 능력""",2021.08.05. 오전 7:27,한국투자증권 보고서한국투자증권은 5일 한국타이어앤테크놀로지에 대해 투자의견 매수를 ...


In [14]:
news_sub.to_csv("../crawl_data/naver_news_content.csv")

#### 기존에 수집한 데이터를 이용한 서브페이지 수집

In [18]:
new_df=pd.read_csv("../crawl_data/naver_news.csv", index_col = 0)

In [19]:
new_df

,section,news_title,link_url
0,헤드라인,"윤석열, 최재형, 홍준표 오늘 이준석 대표 안만난다",https://news.naver.com/main/read.naver?mode=LS...
1,헤드라인,추악한 성추행 피해 11명…추락한 대선후보 쿠오모,https://news.naver.com/main/read.naver?mode=LS...
2,헤드라인,文 “한·미훈련 美와 신중 협의”… 범여(汎與) 의원 60명 ‘연기 연판장’,https://news.naver.com/main/read.naver?mode=LS...
3,헤드라인,"닷새 동안 팔린 김밥 4,200여 줄…130명 넘게 식중독",https://news.naver.com/main/read.naver?mode=LS...
4,헤드라인,"교육부, 4단계에도 등교 검토…9일 등교 방침 발표",https://news.naver.com/main/read.naver?mode=LS...
5,정치,"[최강시사] 이준석 “안철수 ‘전범’ 운운, 정상적인 정치 지도자 간 대화 아냐”",https://news.naver.com/main/read.naver?mode=LS...
6,정치,"사흘간 영남 찾는 최재형, 월성 원전도 방문…윤석열, 나흘간 휴가",https://news.naver.com/main/read.naver?mode=LS...
7,정치,"김근식 “이재명, 음주운전 공격받자 김연경 끌어들여”",https://news.naver.com/main/read.naver?mode=LS...
8,정치,"이준석 ""안철수, 그냥 내가 싫은 거 아닌가?""",https://news.naver.com/main/read.naver?mode=LS...
9,정치,"'도리도리 점검단'…윤석열, '쩍벌' 이어 또 셀프디스",https://news.naver.com/main/read.naver?mode=LS...


In [20]:
# 빈 데이터 프레임 하나 생성
import pandas as pd
dict_sub = {}
dict_sub["기사제목"] = []
dict_sub["기사 입력 시간"] = []
dict_sub["기사 내용"] = []
news_sub_main = pd.DataFrame(dict_sub)
news_sub_main

,기사제목,기사 입력 시간,기사 내용


In [21]:
import time

link_list = new_df["link_url"]
for url in link_list :    
    res = get_news_sub_info(url)
    # 빈 df에 반환되는 dict 행으로 추가(append 함수 이용)
    news_sub_main = news_sub_main.append(res, ignore_index=True)
    time.sleep(2) # 계속되는 요청으로 서버가 disconnect 시킬 수 있으므로 시간 간격을 두어서 요청간격을 조정해야 함
news_sub_main

,기사제목,기사 입력 시간,기사 내용
0,"윤석열, 최재형, 홍준표 오늘 이준석 대표 안만난다",2021.08.05. 오전 8:33,이준석 대표가 지난 7월 29일 오후 서울 여의도 중앙당사에서 열린 국민의힘 대선 ...
1,추악한 성추행 피해 11명…추락한 대선후보 쿠오모,2021.08.05. 오전 5:08,"美뉴욕주 검찰, 5개월간 조사 결과 발표보좌관 성추행·폭로 보복 조치 등 공개CNN..."
2,文 “한·미훈련 美와 신중 협의”… 범여(汎與) 의원 60명 ‘연기 연판장’,2021.08.05. 오전 6:03,"대통령, 軍 주요지휘관 회의 주재서욱 국방에 “여러 가지 고려하라” 김여정 담화 ..."
3,"닷새 동안 팔린 김밥 4,200여 줄…130명 넘게 식중독",2021.08.04. 오후 11:01,동영상 뉴스\n\n\n\n\n\t\n\t[뉴스데스크]◀ 앵커 ▶경기도의 프랜차이즈 ...
4,"교육부, 4단계에도 등교 검토…9일 등교 방침 발표",2021.08.04. 오후 9:51,[헤럴드경제] 신종 코로나바이러스 감염증(코로나19) 4차 대유행이 지속되는 가운데...
5,"[최강시사] 이준석 “안철수 ‘전범’ 운운, 정상적인 정치 지도자 간 대화 아냐”",2021.08.05. 오전 9:30,"- 안철수, 비정상적인 대화 속 긁지 말고, 합당 여부 밝혀야 - 국민의당 지분 요..."
6,"사흘간 영남 찾는 최재형, 월성 원전도 방문…윤석열, 나흘간 휴가",2021.08.05. 오전 9:27,5일 고향 진해 찾고 6일 박정희 생가 방문7일 월성 원전 1호기 방문하고 주민간담...
7,"김근식 “이재명, 음주운전 공격받자 김연경 끌어들여”",2021.08.05. 오전 9:26,“보이콧 주장할 땐 언제고 올림픽 찬양”이재명 더불어민주당 대선 경선 후보가 4일 ...
8,"이준석 ""안철수, 그냥 내가 싫은 거 아닌가?""",2021.08.05. 오전 9:24,[이데일리 박지혜 기자] 국민의힘과 국민의당 합당 논의가 감정싸움으로 번져가는 가운...
9,"'도리도리 점검단'…윤석열, '쩍벌' 이어 또 셀프디스",2021.08.05. 오전 9:22,"반려묘 내세우며 ""'도리도리' 고쳐가겠다""'쩍벌' 논란에는 ""허벅지 살 많아 불편""..."
